# Nyan Cafe Simulation  
Esta es una simulación de una cafetería que consta de ```<nTable>``` mesas, cada una con una capacidad de ```<tableCapacity>``` personas.
Cada ```<spawnTime>``` minutos se generá un grupo de personas de tamaño ```<groupSize>``` que vienen juntas a ocupar una mesa.  
  
Cuando un grupo de personas llega a la cafetería, solicitan una mesa (aleatoriamente); si la mesa no esta disponible, quedan en cola, esperando a que se desocupe la mesa; si ```<groupSize>``` es mayor a ```<tableCapacity>``` el grupo se despacha sin atender; en caso de que la mesa sí este disponible, el grupo pasa a ocuparla.  
  
Cuando el grupo de personas toma una mesa, hacen un pedido de ```<groupSize>``` cafés a un gato robot (seleccionado aleatoriamente), el cúal los atenderá si esta desocupado; si no esta desocupado, los pondrá en cola; en caso de no tener más suministros, despachará a las personas de la mesa y contabilizará como no atendidas.  
  
Cuando el grupo de personas es atendido por el gato robot, este tardará ```<groupSize> X <coffeeTime>``` minutos en preparar sus cafés, luego el gato robot los servirá y pasará a estar desocupado, pero la mesa seguirá ocupada por el grupo de personas por ```<pDrinkingTime>``` minutos más antes de ser desocupada.  
  
Por cada taza de café, los gatos robots gastan ```<gramsPerCoffee>``` gramos de café de su tanque de ```<tankCapacity>``` gramos de capacidad.  
La cafetería tendrá una cantidad de ```<nCatRobot>``` gatos robots funcionando en ella.

## Descripción de variables y enfoque
En este caso se hará enfoque en la **cantidad de mesas vs la cantidad de gatos robot**.  
¿Cómo se comportará la simulación según asignemos mayor o menor cantidad de mesas vs mayor o menor cantidad gatos robot?  
  
Teniendo en cuenta lo anterior, asignaremos ya algunas variables por defecto:

In [31]:
# Importamos las librerias necesarias
import simpy
import pandas as pd
import random

# Definimos algunas variables por defecto
RANDOM_SEED = 42
tableCapacity = 4 # Capacidad de las mesas (Fija a 4 personas)
spawnTime = 40 # Tiempo máximo de llegada de los grupos clientes (1 a 40 minutos)
groupSize = 4 # Tamaño máximo de los grupos de clientes (1 a 4 personas)

pMinDrinkingTime = 10 # Tiempo mínimo de consumo de una taza de café (10 minutos)
pDrinkingTime = 30 # Tiempo máximo de consumo de una taza de café (30 minutos)

coffeeTime = 5 # Tiempo máximo de preparación de una taza de café (1 a 5 minutos)
gramsPerCoffee = 5 # Gasto de café por taza (Fijo a 5 gramos)
tankCapacity = 1000 # Capacidad en gramos del tanque de café de los gatos robots (Fijo a 1000 gramos)

# Definimos los eventos que pueden ocurrir
class Event:
    TOO_MANY_PEOPLE = "TOO_MANY_PEOPLE"
    WAITING_IN_LINE = "WAITING_IN_LINE"
    TABLE_TAKEN = "TABLE TAKEN",
    NOT_ENOUGH_COFFEE = "NOT_ENOUGH_COFFEE"
    MAKING_COFFEE = "MAKING_COFFEE"
    COFFEE_REQ_FINISHED = "COFFEE_REQ_FINISHED"
    DRINKING_COFFEE = "DRINKING_COFFEE"

event = Event()


## Definición de las clases

In [32]:
class Table:
    def __init__(self, env, name, capacity):
        self.env = env
        self.name = name
        self.capacity = capacity # Capacidad de la mesa (En personas) *NO CONFUNDIR CON LA CAPACIDAD DEl RESOURCE*
        self.resource = simpy.Resource(env, capacity=1)

        self.dfT = pd.DataFrame({
            "name": [name],
            "capacity": [capacity]
        })

        self.dfE = pd.DataFrame() # Dataframe para recolectar todos los eventos

    def receiveGroup(self, group):
        print(f"[{self.env.now }] {self.name}: Esta siendo revisada por {group.name}")
        yield self.env.timeout(1)
        if self.capacity < group.size:
            print(f"[{self.env.now}] {self.name}: No tiene suficiente espacio para {group.name} (size:{group.size})")
            df = pd.DataFrame({
                "time": [self.env.now],
                "table": [self.name],
                "group": [group.name],
                "group_size": [group.size],
                "event": [event.TOO_MANY_PEOPLE]
            })
            self.dfE = pd.concat( [ self.dfE , df ] )

        else:
            print(f"[{self.env.now}] {self.name}: Tiene suficiente espacio para {group.name} (size:{group.size})")
            df = pd.DataFrame({
                "time": [self.env.now],
                "table": [self.name],
                "group": [group.name],
                "group_size": [group.size],
                "event": [event.TABLE_TAKEN]
            })
            self.dfE = pd.concat( [ self.dfE , df ] )
            group.hasTable = True



In [33]:
class Group: # Grupo de clientes
    def __init__(self, env, name, size, drinkingTime, table, robocats):
        self.env = env
        self.name = name
        self.size = size
        self.drinkingTime = drinkingTime
        self.table = table
        self.robocats = robocats
        self.hasTable = False
        self.wasAttended = False

        self.dfG = pd.DataFrame({
            "name": [name],
            "size": [size],
            "drinkingTime": [drinkingTime],
            "table": [table.name]
        })

        self.dfE = pd.DataFrame() # Dataframe para recolectar todos los eventos

        env.process(self.in_line())

    def in_line(self): 
        print(f"[{self.env.now}] {self.name}: Esta esperando la mesa {self.table.name}")
        df = pd.DataFrame({
            "time": [self.env.now],
            "table": [self.table.name],
            "group": [self.name],
            "event": [event.WAITING_IN_LINE]
        })
        self.dfE = pd.concat([self.dfE, df])

        with self.table.resource.request() as table_request:
            yield table_request
            print(f"[{self.env.now}] {self.name}: Ha tomado la mesa {self.table.name}")
            yield self.env.process(self.table.receiveGroup(self))
            yield self.env.timeout(1)
            if self.hasTable:
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "table": [self.table.name],
                    "group": [self.name],
                    "event": [event.TABLE_TAKEN]
                })
                self.dfE = pd.concat([self.dfE, df])
                robocat = random.choice( self.robocats )
                print(f"[{self.env.now}] {self.name}: Listo para pedir a {robocat.name}")
                yield self.env.timeout(1)
                yield self.env.process(robocat.makeCoffee(self, self.table))
                if self.wasAttended:
                    print(f"[{self.env.now}] {self.name}: Recibió su café de {robocat.name}")
                    yield self.env.timeout(self.drinkingTime) # Tiempo de consumo de café
                    df = pd.DataFrame({
                        "time": [self.env.now],
                        "table": [self.table.name],
                        "group": [self.name],
                        "event": [event.DRINKING_COFFEE]
                    })
                self.dfE = pd.concat([self.dfE, df])
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "table": [self.table.name],
                    "group": [self.name],
                    "event": [event.COFFEE_REQ_FINISHED]
                })
                self.dfE = pd.concat([self.dfE, df])
                print(f"[{self.env.now}] {self.name}: Abandonó la mesa {self.table.name}")


In [34]:
class RoboCat: # Gato robot
    def __init__(self, env, name, tankCapacity, coffeeTime):
        self.env = env
        self.name = name
        self.tankCapacity = tankCapacity
        self.coffeeTime = coffeeTime
        self.tank = simpy.Container(env, init=tankCapacity, capacity=tankCapacity)

        self.dfR = pd.DataFrame({
            "name": [name],
            "tankCapacity": [tankCapacity],
            "coffeeTime": [coffeeTime]
        })

        self.dfE = pd.DataFrame() # Dataframe para recolectar todos los eventos

    def makeCoffee(self, group, table):
        print(f"[{self.env.now}] {self.name}: Procesando pedido de {group.name} en {group.table.name}")

        totalCoffeeGrams = group.size * gramsPerCoffee

        if self.tank.level < totalCoffeeGrams:
            print(f"[{self.env.now}] {self.name}: No hay suficiente café para {group.name}, {group.table.name}, tanque:{self.tank.level}g, pedido: {totalCoffeeGrams}g")
            df = pd.DataFrame({
                "time": [self.env.now],
                "robocat": [self.name],
                "table": [table.name],
                "group": [group.name],
                "group_size": [group.size],
                "event": [event.NOT_ENOUGH_COFFEE]
            })
            self.dfE = pd.concat( [ self.dfE , df ] )
        else:
            yield self.tank.get(totalCoffeeGrams)
            totalCoffeeTime = group.size * self.coffeeTime
            print( f"[{self.env.now}] {self.name} preparará {group.size} cafés en {totalCoffeeTime} minutos" )
            df = pd.DataFrame({
                "time": [self.env.now],
                "robocat": [self.name],
                "table": [table.name],
                "group": [group.name],
                "group_size": [group.size],
                "event": [event.MAKING_COFFEE]
            })
            self.dfE = pd.concat( [ self.dfE , df ] )

            yield self.env.timeout(totalCoffeeTime)
            group.wasAttended = True


In [35]:
class NyanCafe: # Cafetería
    def __init__(self, env, tableCapacity, spawnTime, groupSize, pMinDrinkingTime, pDrinkingTime, coffeeTime, gramsPerCoffee, tankCapacity, nRobocats, nTables, nGroups):
        self.env = env
        self.tableCapacity = tableCapacity
        self.spawnTime = spawnTime
        self.groupSize = groupSize
        self.pMinDrinkingTime = pMinDrinkingTime
        self.pDrinkingTime = pDrinkingTime
        self.coffeeTime = coffeeTime
        self.gramsPerCoffee = gramsPerCoffee
        self.tankCapacity = tankCapacity
        self.nGroups = nGroups
        self.groups = []

        self.tables = [
            Table(
                env, 
                f"Table {i}", 
                tableCapacity
            ) for i in range(nTables)
        ]
        self.robocats = [
            RoboCat(
                env, 
                f"RoboCat {i}", 
                tankCapacity, 
                coffeeTime
            ) for i in range(nRobocats)
        ]

        env.process(self.spawnGroups())

    def spawnGroups(self):
        for i in range( self.nGroups ):
            size = random.randint(1, self.groupSize)
            drinkingTime = random.randint(self.pMinDrinkingTime, self.pDrinkingTime)
            table = random.choice(self.tables)
            self.groups.append(
                Group(
                    self.env, 
                    f"Group {i}", 
                    size, 
                    drinkingTime, 
                    table, 
                    self.robocats
                )
            )
            yield self.env.timeout(random.randint(1, self.spawnTime))

## Simulación

In [36]:
random.seed(RANDOM_SEED)
env =  simpy.Environment()
nyanCafe = NyanCafe(
    env, 
    tableCapacity, 
    spawnTime, 
    groupSize, 
    pMinDrinkingTime, 
    pDrinkingTime, 
    coffeeTime, 
    gramsPerCoffee, 
    tankCapacity, 
    nRobocats=2, 
    nTables=3, 
    nGroups=5
)
env.run( until=480 ) # 8 horas de simulación


[0] Group 0: Esta esperando la mesa Table 2
[0] Group 0: Ha tomado la mesa Table 2
[0] Table 2: Esta siendo revisada por Group 0
[1] Table 2: Tiene suficiente espacio para Group 0 (size:1)
[2] Group 0: Listo para pedir a RoboCat 0
[3] RoboCat 0: Procesando pedido de Group 0 en Table 2
[3] RoboCat 0 preparará 1 cafés en 5 minutos
[8] Group 0: Recibió su café de RoboCat 0
[18] Group 1: Esta esperando la mesa Table 2
[18] Group 0: Abandonó la mesa Table 2
[18] Group 1: Ha tomado la mesa Table 2
[18] Table 2: Esta siendo revisada por Group 1
[19] Table 2: Tiene suficiente espacio para Group 1 (size:2)
[20] Group 1: Listo para pedir a RoboCat 0
[21] RoboCat 0: Procesando pedido de Group 1 en Table 2
[21] RoboCat 0 preparará 2 cafés en 10 minutos
[25] Group 2: Esta esperando la mesa Table 0
[25] Group 2: Ha tomado la mesa Table 0
[25] Table 0: Esta siendo revisada por Group 2
[26] Table 0: Tiene suficiente espacio para Group 2 (size:4)
[27] Group 2: Listo para pedir a RoboCat 0
[28] RoboCat 